In [ ]:
# Import libraries

import numpy as np
import pandas as pd
from IPython.display import display
from xgboost import XGBRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

___
### Auxiliary functions

In [ ]:
def create_ordinal_ids(df, action_subset='LEAVE'):
    """
    Creates ordinal IDs for persons and clubs based on increasing average
    duration. Set the action_subset on LEAVE (JOIN) to focus on person
    duration IN (OUT of) the club.

    :param df: input dataframe
    :param action_subset: LEAVE/JOIN to focus on IN/OUT club duration
    :return: dataframe augmented with ordinal ID columns
    """
    # add temporary status duration (months) relative to 31/12/2012
    df['status_dur'] = ((df['date'].max() - df['date']) /
                        np.timedelta64(1, 'M'))

    # focus either on person duration in or out of the club
    dft = df[df['action'] == action_subset]

    # define ordinal club IDs as the average duration per club
    dfc = dft.groupby('club_id')['duration'].mean().sort_values()
    dfc = dfc.rename('ord_club_id').reset_index()

    # define ordinal person IDs as the average duration per person
    dfp = dft.groupby('person_id')['duration'].mean().sort_values()
    dfp = dfp.rename('ord_person_id').reset_index()

    # merge ordinal IDs to original input data
    df = pd.merge(df, dfc, on='club_id', how='left')
    df = pd.merge(df, dfp, on='person_id', how='left')

    # # for persons who joined and never left a club or clubs that have only
    # # members where nobody ever left, assign ordinal IDs based on active
    # # status average duration (these IDs will have larger values than the
    # # ordinal IDs already set)
    # df.loc[df['ord_person_id'].isna(), 'ord_person_id'] = df.loc[
    #     df['ord_person_id'].isna(), 'status_dur']
    # df.loc[df['ord_club_id'].isna(), 'ord_club_id'] = df.loc[
    #     df['ord_club_id'].isna(), 'status_dur']

    # drop status duration column
    df = df.drop(columns=['status_dur'])

    return df


def test_train_split(df, first='JOIN', second='LEAVE'):
    """
    Splits the data into testing and training sets.

    The training data contains the full history of complete first/second action
    pairs (sequential JOIN/LEAVE or LEAVE/JOIN pairs) per person-club.
    A mask filters rows that match first/second action sequence pattern.
    Therefore, person-club IDs with an even number of rows are maintained,
    while the last row for the IDs with an odd number of rows is removed.

    The training data include the odd row left from the training data mask.
    They capture a "still active" status based on first action. For instance,
    a) JOIN/LEAVE pattern in training, testing includes most recent JOIN,
    b) LEAVE/JOIN pattern in training, testing includes most recent LEAVE.

    For the testing data nullify the duration column (to be predicted by the
    model) and create a new column that describes the active (deactive) status
    duration from the day of JOIN (LEAVE) until Dec 31st, 2012.

    :param df: input dataframe
    :param first: first action (JOIN for JOIN/LEAVE, or LEAVE for LEAVE/JOIN)
    :param second: second action in pair
    :return: training and testing dataframes
    """
    # # backtest: merge frequencies of person-club IDs to verify
    # # whether the train/test split logic works properly
    # id_freq = df['id'].value_counts().rename('id_freq')
    # id_freq.index.name = 'id'
    # id_freq.reset_index()
    # df = pd.merge(df, id_freq, on='id', how='left')

    # pair length and encoding
    N = len(np.asarray([first, second]))
    d = {first: 0, second: 1}

    # mask creation logic:
    #   1. encode action
    #   2. focus within person-club
    #   3. roll through data row-by-row with a window of N=2 rows and
    #      test for pattern
    #   4. use pd.mask to replace 0s (first action in the pattern) to NaNs
    #   5. backfill 1's (second action) up to N-1=1 rows to get rows of
    #      "complete pairs" with value 1
    #   6. fill NaNs with zeros to indicate rows that lack the pattern
    #   7. convert to bool so that it can be used as a mask in the dataframe
    mask = (df['action'].map(d)
            .groupby(df['id'])
            .rolling(window=N, min_periods=N)
            .apply(lambda x: (x == list(d.values())).all(), raw=False)
            .mask(lambda x: x == 0)
            .bfill(limit=N-1)
            .fillna(0)
            .astype(bool)
            .reset_index(level=0, drop=True))

    # split data into testing and training subsets
    df_train = df[mask].copy()
    df_test = df[~mask].copy()

    # keep only relevant rows in the training set (non-missing
    # duration), otherwise the XGBoost regressor will crash
    df_train = df_train.dropna(subset='duration')

    # add status duration (months) relative to 31/12/2012 in the testing data
    df_test['status_dur'] = ((df['date'].max() - df_test['date']) /
                             np.timedelta64(1, 'M'))

    # nullify duration values from testing data (the models will predict them)
    df_test['duration'] = np.nan

    return df_train, df_test


def xgboost(df_train, df_test):
    """
    Performs XGBoost regression of the expected duration on ordinal IDs.
    The prediction is the expected duration for taking action, i.e. LEAVE
    (JOIN) if currently IN (OUT of) a club.
    
    Use a validation dataset to get OOS model performance, because the
    testing data lack observed duration values.

    :param df_train: training dataframe
    :param df_test: testing dataframe
    :return: testing dataframe with duration predictions from the model
    """
    # split the training dataset into training and validation
    df_val = df_train.sample(frac=0.2, random_state=12)
    df_train = df_train.drop(index=df_val.index)

    # load the boosted tree regressor class
    xgr = XGBRegressor(max_depth=3)

    # fit the model in the training set
    xgr.fit(df_train[['ord_person_id', 'ord_club_id']], df_train['duration'])

    # get the model's duration predictions in the testing set
    yhat = xgr.predict(df_test[['ord_person_id', 'ord_club_id']])

    # validate the model
    R2 = xgr.score(df_val[['ord_person_id', 'ord_club_id']], df_val['duration'])
    print('\n\033[1m' + 'XGBoost Model performance R2 = {}'
          .format(R2.round(2)) + '\033[0m')

    # place the predicted duration in the testing dataframe
    df_test['duration'] = yhat

    return df_test


def create_output(df, action='LEAVE'):
    """
    Shows summary output with join/leave count stats.

    :param df: dataframe with model predictions
    :return: None
    """
    # summary wording and months list
    word = 'departures' if action == 'LEAVE' else 'joins'
    months = [3, 6, 12, 18]

    # estimate remaining time in the club (months)
    df['remaining_time'] = df['duration'] - df['status_dur']

    # form conditions
    cond1 = df.loc[df['remaining_time'] <= 3, 'person_id']
    cond2 = df.loc[df['remaining_time'].between(3, 6, inclusive='right'), 'person_id']
    cond3 = df.loc[df['remaining_time'].between(6, 12, inclusive='right'), 'person_id']
    cond4 = df.loc[df['remaining_time'].between(12, 18, inclusive='right'), 'person_id']

    # show the join/leave actions count within time intervals
    # and their cumulative sum to answer first question
    data = [[cond1.count(), cond2.count(), cond3.count(), cond4.count()]]
    data.append([df.loc[df['remaining_time'] <= m, 'person_id'].count() for m in months])
    out = pd.DataFrame(data, columns=[str(m) + ' mon' for m in months])
    out.index = ['Interval ' + word, 'Total ' + word]
    print('\n\033[1m' + 'Count of people who ' + action +
          ' across various month intervals' + '\033[0m')
    display(out)

    # rank by probability of action in 3, 6, 12, 18 months, where
    # prob = (m - remaining_time) / m, m in (3, 6, 12, 18) months
    for mon in months:
        print('\n\033[1m' + 'Ranking by probability of action ' + action +
              ' in {} months'.format(mon) + '\033[0m')
        mask = df['remaining_time'].between(0, mon, inclusive='right')
        dft = df.loc[mask, ['person_id', 'remaining_time']]
        dft['probability'] = (mon - dft['remaining_time']) / mon
        display(dft.sort_values(by='probability'))

    # predict the action date
    print('\n\033[1m' + 'Predicted date of ' + action + '\033[0m')
    col = action + ' date'
    df[col] = pd.to_datetime('1-1-2013')
    new_leave_date = (df[col] + (df['remaining_time'] * 30)
                      .values.astype('timedelta64[D]'))
    df[col] = np.where(df['remaining_time'] > 0, new_leave_date, df[col])
    display(df[['person_id', 'remaining_time', col]].sort_values(by=col))

___
### Data parsing, processing and feature engineering

The raw data include various types of errors such as duplicate rows, successive "JOIN" rows for the same person-club without leaving (e.g. person=68895, club=1803), and joining and leaving on the same date which adds noise to club membership duration (e.g. person=10, club=1808).

Filtering conditions for each type of error above may still miss or even generate new corner cases that yield ambiguous results with regard to a meaningful sequence of join/leave actions for certain person-clubs. Alternatively, it is more straightforward and less prone to interpretation errors to consider only person-clubs with sequential pairs of "JOIN" followed by "LEAVE" values and allow the possibility of "JOIN" as the last row for persons who are still active in the club as of the last date in the sample. Any person-club in the data that deviates from either
1. complete JOIN/LEAVE historical action pairs or
2. the complete historical pairs with one more JOIN row for currently active membership
is considered to follow an irregular pattern and it is removed from sample. This is a conservative approach, although not the most restrive possible. For instance, a person who follows an acceptable pattern of actions for one club but not for another will still be in the sample.

The presence of dates in the raw data provides for the engineering of a duration metric. The **first key insight** of this analysis is to define a duration metric that can help answers the question but also that can be constructed relatively easy by the data. For instance, one candidate feature is a person's duration in or out of a *particular* club, while another feature is a person's duration in or out of a *any* club. Given the data structure that was enforced according to the filtering guidelines above, the former duration metric is much easier to construct and interpret within a model. This metric choice reinforces the filtering guidelines because certain types of outliers must be removed from the data because they cannot be trained. These outlier cases include the following:
1. Persons who joined one or multiple clubs once and never left any of them.
2. Persons $p$ who joined and left club $c_1$ and then joined club $c_2$ and never left. The rows for person-club $p$-$c_1$ will be part of the sample. However, the unique row for person-club $p$-$c_2$ is dropped.
3. Persons who joined a club first time on relatively later dates (closer to Dec 31, 2012) are a subset of case #1 and are also removed from sample.

The summary statistics for duration reveal a heavy imbalance between non-missing values for duration IN against duration OUT of the club. This is a repercussion of the data structure that is enforced on the data to make the problem tractable. As a result, any candidate model is expected to perform better for the first part of the questions (people leaving a club) than the second (people joining a club).

In [ ]:
# read dataset
df = pd.read_csv("join_the_club.csv").drop(columns=['Unnamed: 0'])

# convert date string to datetime and remove the
# time info from display with the normalize method
df['date'] = pd.to_datetime(df['date']).dt.normalize()

# auxiliary joint identifier that combines person and club IDs
# to group the data and simplifies the filtering code
df['id'] = df['person_id'].astype(str) + '_' + df['club_id'].astype(str)

# drop duplicates across all columns and sort by person, club, date and action.
# including action in the sort simplifies the sanity checks below, because
# "JOIN" precedes "LEAVE" alphabetically.
cols = ['person_id', 'club_id', 'date', 'action']
df = df.drop_duplicates().sort_values(by=cols)

# group data by person-club and create an auxiliary series
# with the previous action within the group.
s = df.groupby('id')['action'].shift()

# # verify that the first action within each group is "JOIN".
# # instead of using groupby.first(), it is computationally more
# # efficient to extract the rows where the shifted series is NaN.
# f = df.loc[s.isna(), 'action'].value_counts()
# print('Distribution of first action within person-club groups\n', f)

# create boolean column that is true if there was a change in
# action between two successive rows or if the row is the first
# within the person-club group. If there is at least one False value
# within a person-club group, it implies a data irregularity that is
# assumed to be a data error.
df['valid'] = (s.ne(df['action']) | s.isna())

# remove the person-club groups that have at least one error,
# even if the bulk of the sequence satisfies the sanity criteria
drop_ids = df.loc[~df['valid'], 'id'].unique()
df = df[~df['id'].isin(drop_ids)].drop(columns=['valid'])

# remove the person-club groups with only one observation (persons
# who joined a particular club and never left, no training history)
outlier_ids = df['id'].value_counts()
outlier_ids = outlier_ids[outlier_ids == 1].index.tolist()
df = df[~df['id'].isin(outlier_ids)]

# person duration in and out of each club (months)
df['duration'] = df.groupby('id')['date'].diff() / np.timedelta64(1, 'M')

# save a copy of the full data and provide summary statistics
print('\033[1m' + 'Summary Statistics' + '\033[0m')
data = df.copy()
data.to_csv('parsed_data.csv', index=False)
display(data.astype(str).drop(columns=['date', 'duration']).describe())

s1 = (data.loc[df['action'] == 'JOIN', 'duration']
      .describe().rename('duration out of club'))
s2 = (data.loc[df['action'] == 'LEAVE', 'duration']
      .describe().rename('duration in club'))
display(pd.concat([s1, s2], axis=1).T)

___
### Modeling (part 1): people leaving clubs

#### Key metric

The list of features includes person, club and person-club IDs, a numeric person-club duration in/out of the club, and a categorical action. The **second key insight** of this analysis is to use the duration as the dependent variable instead of using it as a feature to predict the categorical action. One difficulty of a categorical dependent variable is that the indicator function (e.g. logistic function for a logistic regression) will never achieve numerically a value of exactly 1 to answer questions such as "what persons are leaving (with 100% probability) in 3 months". Therefore, the goal of the model is to predict a person's duration of staying in/out of a particular club. This expected duration can be combined with the time that a person is already in/out of a club as of Dec 31st, 2012 to provide answers to the problem's questions. In addition, varying durations across persons help define a probability measure for leaving/joining a club within 3, 6 or more months as of the reference date.

#### Ordinal person and club IDs

The action feature is used to distinguish the cases of duration in vs duration out of the club. As a result, the remaining features are categorical IDs. In order to feed them to the model as dependent variables, it is necessary to encode them first. One-hot encoding as dummies would result in a very sparse dataset, large estimation time and potentially algorithm convergence failures. Therefore, I introduce a duration-based ordinal encoding.

The ordinal club ID for a particular club is the average duration across all persons who have joined that club. Similarly, the ordinal person ID is the person's average duration across all clubs joined and then left. The intuition is that persons with large historical duration in clubs will tend to stay longer than others in any new club joined. Also, clubs where members used to stay longer on average will still host new members for longer periods relative to other clubs.

Notice that ordinal ID ties are possible and they don't create any issues with interpretability. For instance, two clubs in the sample have zero average duration (all members have joined and left on the same day). Whatever the model prediction is for the common ordinal club ID, it can be mapped back to both clubs by means of their unique club ID that was provided in the raw data. Similarly, many persons can have the same expected duration, each for a different club, without any issues of mapping the same prediction back to their unique person ID.

#### Training/validation/testing data split and model fit

The training data contains the full history of complete JOIN/LEAVE pairs per person-club. A mask filters rows that match JOIN/LEAVE sequence pattern. Therefore, person-club IDs with an even number of rows are maintained, while the last row for the IDs with an odd number of rows is removed.

The training data include the odd row left from the training data mask. That data captures a "still active" status since the person JOINed the club. The testing subset also includes a column that describes the active status duration from the day of JOIN until Dec 31st, 2012.

The model extracts a validation sample from the training dataset to estimate an out-of-sample model performance metric, because the testing data don't have any observed durations to be compared with the model predicted values. The model includes a boosted tree regression, because of the small number and nonlinearities of the features.

#### Model output

The model prediction on the duration is independent of time. The *"remaning"* time in a club is defined as the person's model-predicted duration minus his months in the club as of Dec 31, 2012. This variable determines the person's expected action according to the model with the following logic:
1. If the remaining time is positive, it shows how long that person has in a particular club. This can determine what persons leave by the 3, 6, 12 and 18-month thresholds.
2. If the remaining time is negative, the model predicts that this person should have left the club already or will leave immediately. This possibility may lead to biased results and highlights a limitation to the model (see below).

A person's probability of leaving by a certain horizon $m = \{3, 6, 12, 18\}$ months is $prob = (m - r) / m$, where $r$ is the person's remaining time. The exact date of departure is also specified by the remaining time as of Dec 31, 2012.

#### Results and model limitations

The model has an accuracy of 61% for the case of LEAVE predictions. The summary statistics table on the people leaving shows a relatively larger amount of departures within 3 months compared to longer intervals. The reason for this bias is that there are plenty of persons with a history of small duration in clubs who have also joined one more club but never left it. The model is biased to predict a small duration for these persons, but because the person has already stayed too long, the remaining time is negative.

The model implies that these persons should have left a long time ago (i.e. it failed to predict the non-departure) or that they should leave immediately. This is a model limitation that cannot be mitigated, because the person's habit in the training set is to leave the club relatively fast, while in the training set the preference has changed to stay for the long term (perhaps indefinitely).

The probability of leaving is contrasted with the remaining time. The tables clearly indicate that persons with remaining time close to the 3, 6, 12 or 18-month thresholds are less likely to leave. On the contrary, those with a remaining time much smaller than the threshold are more likely to leave during that period.

Finally, the predicted date of departure is a transformation of the remaining time (in months) relative to the reference date Dec 31, 2012. This table highlights the bias of excessive number of negative remaining time, where according to the model the departure date is effective immediately.

In [ ]:
# # read the parsed data
# df = pd.read_csv("parsed_data.csv")

# define ordinal person and club IDs from person duration IN the club
df = create_ordinal_ids(data, action_subset='LEAVE')

# split main data into training and testing subsets
df_train, df_test = test_train_split(df, first='JOIN', second='LEAVE')

# fill in the model-predicted durations in the testing dataset
dfp = xgboost(df_train, df_test)

# show results
create_output(dfp, action='LEAVE')

___
### Modeling (part 2): people joining clubs

The same logic applies for the creation of ordinal IDs, training/validation/testing data splits and model fit. The difference is that the JOIN/LEAVE patter for the training set is not reversed to LEAVE/JOIN to focus on duration **out** of the club.

The model performance is worse compared to part 1 of the analysis because of the enforced structure of JOIN/LEAVE patterns in the processed data. This structure implies much fewer LEAVE/JOIN pairs that are used in part 2. This data structure also biases the estimator toward persons who don't stay long outside of a club. Therefore, it is no surprise that the model performance has dropped to less than half and the number of predicted joins within less than 3 months is even larger than the corresponding biased number of departures during the same interval.

In [ ]:
# # read the parsed data
# df = pd.read_csv("parsed_data.csv")

# define ordinal person and club IDs from person duration OUT of the club
df = create_ordinal_ids(data, action_subset='JOIN')

# split main data into training and testing subsets
df_train, df_test = test_train_split(df, first='LEAVE', second='JOIN')

# fill in the model-predicted durations in the testing dataset
dfp = xgboost(df_train, df_test)

# show results
create_output(dfp, action='JOIN')